In [39]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:97% !important; }</style>"))

import jupyternotify
ip = get_ipython()
ip.register_magics(jupyternotify.JupyterNotifyMagics)

import scoring as score # para hacer los reportes de puntajes
from time import time

from sklearn import preprocessing
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler  
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV, RandomizedSearchCV, ShuffleSplit

properati = pd.read_csv('datos/caba_para_mapa.csv',index_col=0,error_bad_lines=False)

#tenia un nan nose porque
properati.dropna(inplace=True)

<IPython.core.display.Javascript object>

In [40]:
#paso features con strings a numericos
le_barrio = preprocessing.LabelEncoder()
barrios=properati['place_name']
le_barrio.fit(barrios)
properati['place_name'] = le_barrio.transform(barrios)

le_divisa = preprocessing.LabelEncoder()
divisa=properati['currency']
le_divisa.fit(divisa)
properati['currency'] = le_divisa.transform(divisa)

le_tipo = preprocessing.LabelEncoder()
tipos_prop=properati['property_type']
le_tipo.fit(tipos_prop)
properati['property_type'] = le_tipo.transform(tipos_prop)

### Pruebo con np.histogram

In [41]:
cant_buckets = 6

#buckets
cantidad,rango = np.histogram(properati['price_usd_per_m2'], bins=cant_buckets)
properati['categories_by_price']=pd.cut(properati['price_usd_per_m2'],rango,labels=np.arange(cant_buckets))
properati['price_range']=pd.cut(properati['price_usd_per_m2'],rango)
#lo casteo a float porque si no tira error 
properati['categories_by_price']=properati['categories_by_price'].astype(np.float64) 

#tenia un nan nose porque
properati.dropna(inplace=True)

properati.groupby('categories_by_price').count()

,property_type,place_name,state_name,lat-lon,lat,lon,price,currency,price_aprox_local_currency,price_aprox_usd,...,expenses,properati_url,description,title,dist_a_subte,dist_a_tren,dist_a_univ,dist_a_villa,dist_a_zona_anegada,price_range
categories_by_price,,,,,,,,,,,,,,,,,,,,,
0.0,19,19,19,19,19,19,19,19,19,19,...,19,19,19,19,19,19,19,19,19,19
1.0,795,795,795,795,795,795,795,795,795,795,...,795,795,795,795,795,795,795,795,795,795
2.0,1272,1272,1272,1272,1272,1272,1272,1272,1272,1272,...,1272,1272,1272,1272,1272,1272,1272,1272,1272,1272
3.0,293,293,293,293,293,293,293,293,293,293,...,293,293,293,293,293,293,293,293,293,293
4.0,62,62,62,62,62,62,62,62,62,62,...,62,62,62,62,62,62,62,62,62,62
5.0,15,15,15,15,15,15,15,15,15,15,...,15,15,15,15,15,15,15,15,15,15


In [42]:
#preparo set de datos
X = zip(properati['dist_a_subte'],properati['dist_a_univ'],properati['dist_a_tren'],properati['dist_a_villa'],\
        properati['dist_a_zona_anegada'],properati['surface_total_in_m2'],properati['place_name'], \
        properati['property_type'])
y = properati['categories_by_price']

### Pruebo con Quantile . NO MEJORO saltear

In [4]:
from sklearn.preprocessing import QuantileTransformer
scaler = QuantileTransformer(copy=False)

In [5]:
#preparo set de datos
X = zip(properati['dist_a_subte'],properati['dist_a_univ'])
y = properati['price_usd_per_m2']
print X[0]
X=scaler.fit_transform(X,y)
print X[0]

#reviso si quedo datos anomalos
print (np.any(np.isnan(y)) == False)
print (np.all(np.isfinite(y)) == True )

(7.0, 8.5)
[ 0.86886887  0.86636637]
True
True


### Pruebo con RobustScaler. NO MEJORO saltear

In [37]:
from sklearn.preprocessing import RobustScaler
scaler = RobustScaler(copy=False)

In [38]:
#preparo set de datos
X = zip(properati['dist_a_subte'],properati['dist_a_univ'])
y = properati['price_usd_per_m2']
print X[0]
X=scaler.fit_transform(X,y)
print X[0]

#reviso si quedo datos anomalos
print (np.any(np.isnan(y)) == False)
print (np.all(np.isfinite(y)) == True )

(7.0, 8.5)
[ 1.1875      0.97560976]
True
True


### Busco hiper-parametros con grid search

In [48]:
%%notify

from sklearn.model_selection import train_test_split
rf = RandomForestClassifier(n_jobs=-1)

bootstrap=[False]
max_features =["auto","sqrt","log2"]
n_estimators = np.arange(10,50,10)

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2)

param_grid = {"bootstrap": bootstrap,"max_features":max_features,"n_estimators":n_estimators}

grid_search = GridSearchCV(rf, param_grid=param_grid ,cv=10,refit=True) #refit deja el estimador fiteado con los mejores hiperparametros
start = time()
grid_search.fit(X_train, y_train)

print("GridSearchCV duro %.2f segundos para %d candidatos a hyper-parametros."
    % (time() - start, len(grid_search.cv_results_['params'])))
print("")
score.report_single(grid_search.cv_results_)

GridSearchCV duro 55.97 segundos para 12 candidatos a hyper-parametros.

Puesto: 1
Promedio training score: 0.993 (std: 0.001)
Promedio validation score: 0.757 (std: 0.029)
Promedio fit time: 0.239s
Hiper-parametros: {'max_features': 'sqrt', 'n_estimators': 30, 'bootstrap': False}

Puesto: 2
Promedio training score: 0.993 (std: 0.001)
Promedio validation score: 0.756 (std: 0.029)
Promedio fit time: 0.170s
Hiper-parametros: {'max_features': 'auto', 'n_estimators': 10, 'bootstrap': False}

Puesto: 3
Promedio training score: 0.993 (std: 0.001)
Promedio validation score: 0.752 (std: 0.034)
Promedio fit time: 0.231s
Hiper-parametros: {'max_features': 'log2', 'n_estimators': 40, 'bootstrap': False}

Puesto: 4
Promedio training score: 0.993 (std: 0.001)
Promedio validation score: 0.752 (std: 0.027)
Promedio fit time: 0.218s
Hiper-parametros: {'max_features': 'auto', 'n_estimators': 40, 'bootstrap': False}

Puesto: 5
Promedio training score: 0.993 (std: 0.001)
Promedio validation score: 0.749 

<IPython.core.display.Javascript object>

In [55]:
grid_search.score(X_test,y_test) # pruebo el estimador refiteado con el set de test

0.74796747967479671